In [1]:
# Import required libraries
import os
import numpy as np
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
from tqdm import tqdm
from torch import optim
import torchvision
from PIL import Image
from Unet import UNet
from Diffusion_model import Diffusion
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

In [3]:
import os
import numpy as np

directory = '../../00_data/Samples'
numpy_list = []

for file_name in os.listdir(directory):
    if file_name.endswith('.npy'):
        file_path = os.path.join(directory, file_name)
        image_np = np.load(file_path)

        numpy_list.append(image_np)



In [4]:


def save_images(generated, path, **kwargs):
    # Convert to numpy arrays
    generated = generated.to('cpu').numpy()

    # Determine the number of images
    num_images = generated.shape[0]
    # Calculate the number of rows needed (assuming 4 images per row for display)
    num_rows = (num_images + 3) // 4  # Round up division

    # Create a figure and axes
    fig, axes = plt.subplots(nrows=num_rows, ncols=4, figsize=(20, num_rows * 5))
    axes = axes.flatten()  # Flatten the axes array for easier indexing15

    # Adjust space between images
    plt.subplots_adjust(wspace=0.3, hspace=0.3)

    for i, gen_img in enumerate(generated):
        # Display generated images
        axes[i].imshow(gen_img.squeeze(), cmap='gray', **kwargs)
        axes[i].axis('off')
        axes[i].set_title(f"Generated {i+1}", fontsize=10)

    # In case the last row is not fully populated
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    # Save the figure
    fig.savefig(path, bbox_inches='tight')
    plt.close(fig)


In [5]:
from PIL import Image

In [6]:

x_train = np.array(numpy_list)
# Convert to a list of PIL images, resize each, and then convert back to a NumPy array
x_train = np.array([np.array(Image.fromarray(img.squeeze()).resize((64, 64))) for img in x_train])

# The new shape will be (10000, 64, 64). If you need to add the channel dimension back:
x_train = x_train[:, np.newaxis, :, :]
x_train = torch.Tensor(x_train)

print(x_train.shape)



torch.Size([10000, 1, 64, 64])


In [7]:
type(x_train)

torch.Tensor

In [8]:
dataset = TensorDataset(x_train)
dataloader = DataLoader(dataset, batch_size=10)


In [9]:
dataloader

In [19]:
device = "cuda"
model = UNet().to(device)
#model.load_state_dict(torch.load('./Weights/Diff_ckpt_1.pt'))
optimizer = optim.AdamW(model.parameters(), lr=3e-4)
mse = nn.MSELoss()
diffusion = Diffusion(img_size=64, device=device)
l = len(dataloader)

In [20]:
start_epoch = 13
epochs = 100

#if model exists, load it
if os.path.exists('./Weights/Diff_ckpt_1.pt'):
    model.load_state_dict(torch.load('./Weights/Diff_ckpt_1.pt'))
    print("Model loaded")


Model loaded


In [21]:
for epoch in range(start_epoch, epochs):
    print(f"Starting epoch {epoch}:")
    pbar = tqdm(dataloader)
    avg_mse = 0
    # for i, (images, conditions) in enumerate(pbar):
    for i, (images,) in enumerate(pbar): #removed conditions for now to make the model DDPM
        images = images.to(device)
        # conditions = conditions.to(device)
        t = diffusion.sample_timesteps(images.shape[0]).to(device)
        x_t, noise = diffusion.noise_images(images, t)
        # predicted_noise = model(x_t, t, conditions)
        predicted_noise = model(x_t, t)
        loss = mse(noise, predicted_noise)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pbar.set_postfix(MSE=loss.item())
        avg_mse += loss.item()

    print(f'Average MSE: {avg_mse/1000:.5f}\n')
    sampled_images = diffusion.sample(model, n=images.shape[0])
    save_images(sampled_images, os.path.join("Results", f"{epoch}.png"))
    torch.save(model.state_dict(), os.path.join("Weights", f"Diff_ckpt_1.pt"))

Starting epoch 13:


100%|██████████| 1000/1000 [02:03<00:00,  8.11it/s, MSE=0.00151]


Average MSE: 0.00186

Sampling 10 new images....


999it [00:44, 22.34it/s]


Starting epoch 14:


...

999it [00:38, 26.16it/s]


Starting epoch 99:


100%|██████████| 1000/1000 [01:54<00:00,  8.76it/s, MSE=0.000923]


Average MSE: 0.00113

Sampling 10 new images....


999it [00:38, 26.25it/s]


In [22]:
from PIL import Image
import numpy as np
import os

def save_images_individually(generated, directory):
    # Convert to numpy arrays
    generated = generated.detach().cpu().numpy()

    # Determine the number of images
    num_images = generated.shape[0]
    
    # Ensure the directory exists
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Save each image individually
    for i, gen_img in enumerate(generated):
        # Remove the channel dimension if it's 1 (for grayscale images)
        if gen_img.shape[0] == 1:
            gen_img = gen_img.squeeze(0)
        # Convert to 0-255 range and uint8 type
        gen_img = ((gen_img - gen_img.min()) * (1/(gen_img.max() - gen_img.min()) * 255)).astype('uint8')
        img = Image.fromarray(gen_img)
        img.save(os.path.join(directory, f"generated_{i+1}.png"))

In [23]:

from tqdm.auto import tqdm
import os

# Load model weights
if os.path.exists('./Weights/Diff_ckpt_1.pt'):
    model.load_state_dict(torch.load('./Weights/Diff_ckpt_1.pt'))
    print("Model loaded")
model.to(device)

# Set the model to evaluation mode
model.eval()

# Generate and save images
with torch.no_grad():
    for i in tqdm(range(100)):
        sampled_images = diffusion.sample(model, n=10)
        # save_images function needs to be defined, as well as a method to name each batch of generated images uniquely
        save_images_individually(sampled_images, os.path.join("Results_individual", f"generated_{i}.png"))




/home/harsh/anaconda3/envs/pytorch-cuda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model loaded


  0%|          | 0/100 [00:00<?, ?it/s]

Sampling 10 new images....


999it [00:39, 25.58it/s]
  1%|          | 1/100 [00:39<1:04:35, 39.15s/it]

...

999it [00:38, 25.64it/s]
 98%|█████████▊| 98/100 [1:03:44<01:17, 38.74s/it]

Sampling 10 new images....


999it [00:39, 25.28it/s]
 99%|█████████▉| 99/100 [1:04:24<00:39, 39.01s/it]

Sampling 10 new images....


999it [00:39, 25.07it/s]
100%|██████████| 100/100 [1:05:04<00:00, 39.04s/it]
